In [28]:
cd /Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/lirpa_iou_pipeline

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/lirpa_iou_pipeline


In [29]:
# Etape 1 : Faire une sélection de 100 images avec une répartition des classes MNIST (10 images/class)
# Etape 2: choisir un pas pour la brightness (on te laisse choisir ce qui te paraît le mieux pour avoir une belle comparaison de vanilla et iou extension)
# Etape 3 Lancer le benchmark pour 100 valeurs de brightness entre [0, …, 0.5]
# Etape 4: faire le setup pour le contrast: tensor_init avec des 1 au lieu de 0. Si tu as un doute, on en discute. Comme pour la brightness tu nous faire un screenshot de ton code
# Visualisation !

In [40]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle
import torch 
import torch.nn.functional as F
from torch import nn
import pandas as pd
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm
from iou_calculator.utils import Merge, show_im, show_im_origin, check_box, clip_corner



In [41]:
cd ..

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip


In [42]:
cd utils

[Errno 2] No such file or directory: 'utils'
/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip


In [43]:
from model_func import *
from torch.utils.data import DataLoader
from PIL import Image, ImageDraw
import plotly.express as px


In [55]:
cd /Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline


In [8]:
class NeuralNetwork_BrightnessContrast(nn.Module):
    '''
    New model that takes as input a brightness or constrat value and apply it to a specific image
    using the linear_perturbation layer
    (aka the weight and biases of the linear_perturbation layer are set to specific values to encode
    brightness or contrast for a specific image)
    '''
    def __init__(self, classif=True):
        super(NeuralNetwork_BrightnessContrast, self).__init__()
        self.classif=classif
        seed = 0
        torch.manual_seed(seed)
        padding = 1
        self.linear_perturbation = nn.Linear(1,90*90)
        self.conv0 = nn.Conv2d(1, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool0 = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Conv2d(16, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool1 = nn.MaxPool2d(2, stride=2)
        self.flatten = nn.Flatten()
        self.linear_relu_stack= nn.Linear(7744,256)
        if self.classif:
            self.linear = nn.Linear(256, 10)
        else:
            self.linear_all = nn.Linear(256, 4) 

    
    def forward(self, alpha):
        layer = self.linear_perturbation(alpha)
        layer = layer.view((-1, 1, 90, 90))
        layer = self.conv0(layer)
        layer = F.relu(self.pool0(layer))
        layer = self.conv1(layer)
        layer = F.relu(self.pool1(layer))
        layer = self.flatten(layer)
        layer = self.linear_relu_stack(layer)
        layer = F.relu(layer)
        if self.classif:
            logits = self.linear(layer)
        else:
            logits = self.linear_all(layer)
        return logits

## Digit model

In [9]:
model_digit = NeuralNetwork_BrightnessContrast(classif=True)

filename = 'toy_model_classif'
model_torch_load  = torch.jit.load(f'{filename}.pt')
digit_config = model_torch_load.state_dict()
digit_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
digit_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_digit.load_state_dict(digit_config)
#model_torch_load.state_dict()

<All keys matched successfully>

## Corner Model

In [10]:
model_corners = NeuralNetwork_BrightnessContrast(classif=False)
filename = 'toy_model_corners'
model_torch_load  = torch.jit.load(f'{filename}.pt')
corner_config = model_torch_load.state_dict()
corner_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
corner_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_corners.load_state_dict(corner_config)
#model_torch_load.state_dict()


<All keys matched successfully>

## Load Data

In [11]:
cd utils

/Users/ncohmpli/Documents/Professionnel/Technical/Verification/Lirpa for IoU/manip/pipeline/utils


In [12]:
train_df = pd.read_csv("train.csv")
dict_df_10 = {}
for label in np.unique(list(train_df["label"])): 
    dict_df_10[label] = train_df[train_df["label"] == label].head(10)

In [13]:
train_df_100 = pd.concat(dict_df_10.values())
trainingData = CustomMnistDataset_OL(train_df_100)
train_dataloader = DataLoader(trainingData, batch_size=1, shuffle=False)
iterator = iter(train_dataloader)
print(len(train_dataloader))

100


In [14]:
trainingData.df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
71,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
data = next(iter(train_dataloader))
X_0, _ = data
data = next(iter(train_dataloader))
X_1, Y_1 = data

In [16]:
Y_1

[tensor([0]), tensor([], size=(1, 0))]

## Brightness

brightness(x; alpha)= x+ alpha

thus for linear_perturbation

linear_perturbation.weight = [1..1]

linear_perturbation.weight = x

In [17]:
def set_brightness(model, image):
    # create a pytorch model for a specific flatten image
    # once a model has been set for one images, you can call it for any brightness value
    image_flatten = image.view((90*90,))
    brightness_config = model.state_dict()
    brightness_config['linear_perturbation.bias']=image_flatten
    brightness_config['linear_perturbation.weight']=torch.Tensor(np.ones((90*90,1), dtype='float32'))
    model.load_state_dict(brightness_config)
    

## AutoLirpa on brightness

In [18]:
data = next(iterator)

In [19]:
X, y = data

In [20]:
variations_brightness=np.linspace(0, 0.00560,5)
variations_brightness

array([0.    , 0.0014, 0.0028, 0.0042, 0.0056])

In [21]:
def bound_brightness(df=train_df_100[:3], variations_brightness=np.linspace(0, 0.00560,101)): 
    tensor_init_brightness = torch.tensor([[0.0]]).float().to('cpu')
    dico_brightness = {"gt_box":[], "image_id":[], "alpha_range":[], "brightness_variation":[], "lb":[], "ub":[]}
    
    for image_id in range(len(df)):
            model_lirpa_corners = BoundedModule(model_corners, tensor_init_brightness)
            data = next(iterator)
            X, y = data
            set_brightness(model_corners, X)
            gt_logit, gt_box = y
            gt_box = gt_box.detach().numpy()[0]
            for brightness_variations in variations_brightness:
                ptb_brightness = PerturbationLpNorm(norm=np.inf, eps=brightness_variations) #attention espilon c'est X_brightness
                input_lirpa_brightness = BoundedTensor(tensor_init_brightness, ptb_brightness)
                lb_brightness, ub_brightness = model_lirpa_corners.compute_bounds(x=(input_lirpa_brightness,),
                                                                    IBP=True, method='crown')
                lb_box = [clip_corner(corner) for corner in lb_brightness[0].tolist()]
                ub_box = [clip_corner(corner) for corner in ub_brightness[0].tolist()]
                print("lb box", lb_box, "ub box", ub_box)
                dico_brightness["gt_box"].append(gt_box)
                dico_brightness["image_id"].append(image_id)
                dico_brightness["alpha_range"].append(str([tensor_init_brightness.item()-brightness_variations,
                                                           tensor_init_brightness.item()+brightness_variations])) 
                dico_brightness["brightness_variation"].append(brightness_variations)
                dico_brightness["lb"].append(lb_box)
                dico_brightness["ub"].append(ub_box)
    df = pd.DataFrame(dico_brightness)
    return(df)

df_brightness = bound_brightness()

lb box [60.90700912475586, 3.8689661026000977, 89.59930419921875, 32.60240173339844] ub box [60.90700912475586, 3.8689661026000977, 89.59930419921875, 32.60240173339844]
lb box [60.74323654174805, 3.7768075466156006, 89.37776184082031, 32.47275161743164] ub box [61.484596252441406, 4.0373311042785645, 90, 33.07496643066406]
lb box [60.563507080078125, 3.675539970397949, 89.13929748535156, 32.3326530456543] ub box [62.10990905761719, 4.239653587341309, 90, 33.59622573852539]
lb box [60.36543273925781, 3.5642218589782715, 88.88118743896484, 32.18040084838867] ub box [62.770179748535156, 4.459465503692627, 90, 34.149085998535156]
lb box [60.15690994262695, 3.4435181617736816, 88.6197509765625, 32.02482604980469] ub box [63.503517150878906, 4.711387634277344, 90, 34.786067962646484]
lb box [59.93177795410156, 3.303096055984497, 88.3404541015625, 31.85071563720703] ub box [64.29476928710938, 4.988986968994141, 90, 35.48591995239258]
lb box [59.71731948852539, 3.1691112518310547, 88.07222747

In [22]:
df_brightness

,gt_box,image_id,alpha_range,brightness_variation,lb,ub
0,[],0,"[0.0, 0.0]",0.000000,"[60.90700912475586, 3.8689661026000977, 89.599...","[60.90700912475586, 3.8689661026000977, 89.599..."
1,[],0,"[-5.6e-05, 5.6e-05]",0.000056,"[60.74323654174805, 3.7768075466156006, 89.377...","[61.484596252441406, 4.0373311042785645, 90, 3..."
2,[],0,"[-0.000112, 0.000112]",0.000112,"[60.563507080078125, 3.675539970397949, 89.139...","[62.10990905761719, 4.239653587341309, 90, 33...."
3,[],0,"[-0.000168, 0.000168]",0.000168,"[60.36543273925781, 3.5642218589782715, 88.881...","[62.770179748535156, 4.459465503692627, 90, 34..."
4,[],0,"[-0.000224, 0.000224]",0.000224,"[60.15690994262695, 3.4435181617736816, 88.619...","[63.503517150878906, 4.711387634277344, 90, 34..."
...,...,...,...,...,...,...
298,[],2,"[-0.005376, 0.005376]",0.005376,"[0, 0, 0, 0]","[90, 90, 90, 90]"
299,[],2,"[-0.005432, 0.005432]",0.005432,"[0, 0, 0, 0]","[90, 90, 90, 90]"
300,[],2,"[-0.005488, 0.005488]",0.005488,"[0, 0, 0, 0]","[90, 90, 90, 90]"
301,[],2,"[-0.005544, 0.005544]",0.005544,"[0, 0, 0, 0]","[90, 90, 90, 90]"


In [23]:
dictio = {"chou":2, "chien":3}

In [26]:
for i, u in dictio.items():
    print(i,u)

chou 2
chien 3


In [27]:
L = [3,4,6]
pd.DataFrame(L)

,0
0,3
1,4
2,6


In [35]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle


import torch 
import torch.nn.functional as F
from torch import nn
import pandas as pd

In [82]:
class NeuralNetwork_BrightnessContrast(nn.Module):
    '''
    New model that takes as input a brightness or constrat value and apply it to a specific image
    using the linear_perturbation layer
    (aka the weight and biases of the linear_perturbation layer are set to specific values to encode
    brightness or contrast for a specific image)
    '''
    def __init__(self, classif=True):
        super(NeuralNetwork_BrightnessContrast, self).__init__()
        
        self.classif=classif
        seed = 0
        torch.manual_seed(seed)
        padding = 1
        
        # create a linear layer that can be used either for brightness or contrast
        self.linear_perturbation = nn.Linear(1,90*90)
        
        self.conv0 = nn.Conv2d(1, 16, 3, padding= padding) # 3x3 filters w/ same padding
        self.pool0 = nn.MaxPool2d(2, stride=2)
        #output shape : 15x15x16
        self.conv1 = nn.Conv2d(16, 16, 3, padding= padding) # 3x3 filters w/ same padding
        
        #print("before self.conv1.weight\n",a)

        
        # Initializing the weights with the Xavier initialization method (equivalent to tf.keras.Initializers.glorot_uniform)
        #nn.init.xavier_uniform(self.conv1.weight)
        #print("after self.conv1.weight\n", b)

        #output shape : 17x17x16 ((15 - 3 + 2*2)/1 + 1 )
         #initialize to zeros 
        #nn.init.zeros_(self.conv1.weight)

        self.pool1 = nn.MaxPool2d(2, stride=2)
        #output shape : 8x8x16
        self.flatten = nn.Flatten()
        #output shape : 1024
        #HERE CHECK RIGHT SIZE FROM FLATTEN TO LINEAR

        self.linear_relu_stack= nn.Linear(7744,256)
        if self.classif:
            self.linear = nn.Linear(256, 10)
        else:
            self.linear_all = nn.Linear(256, 4) 
        """
        self.linear_x = nn.Linear(256, 1)
        self.linear_y = nn.Linear(256, 1)
        self.linear_all = nn.Linear(256, 2) 
        """
        #modifiy here, logits layer must return label, coordinates xmin,ymin,xmax,ymax
        #self.linear_all = nn.Linear(256, 4) 
    
    def forward(self, x):
        x = self.linear_perturbation(x)
        x = x.view((-1, 1, 90, 90))
        x = self.conv0(x)
        x = F.relu(self.pool0(x))
        x = self.conv1(x)
        x = F.relu(self.pool1(x))
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        x= F.relu(x)
        if self.classif:
            logits = self.linear(x)
        else:
            logits = self.linear_all(x)

        return logits#logits, x_min, y_min, x_max, y_max

In [83]:
model_digit = NeuralNetwork_BrightnessContrast(classif=True)

filename = 'toy_model_classif'
model_torch_load  = torch.jit.load(f'{filename}.pt')
digit_config = model_torch_load.state_dict()
# we create values for the linear_perturbation parameters that will me set for brightness and contrast afterwards
# if we don't create the keys in the dictionary, there is a key mismatch with the params dictionary of model_digit
# that raise an error
# For Noemie: you can try to comment the two next lines to see the error by yourself 
digit_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
digit_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_digit.load_state_dict(digit_config)
#model_torch_load.state_dict()

<All keys matched successfully>

In [84]:
model_corners = NeuralNetwork_BrightnessContrast(classif=False)

filename = 'toy_model_corners'
model_torch_load  = torch.jit.load(f'{filename}.pt')
corner_config = model_torch_load.state_dict()
# we create values for the linear_perturbation parameters that will me set for brightness and contrast afterwards
corner_config['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90, 1), dtype='float32'))
corner_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_corners.load_state_dict(corner_config)
#model_torch_load.state_dict()


<All keys matched successfully>

In [86]:
model_corners_contrast = NeuralNetwork_BrightnessContrast(classif=False)

filename = 'toy_model_corners'
model_torch_load_contrast  = torch.jit.load(f'{filename}.pt')
corner_config_contrast = model_torch_load_contrast.state_dict()
# we create values for the linear_perturbation parameters that will me set for brightness and contrast afterwards
corner_config_contrast['linear_perturbation.weight']=torch.Tensor(np.zeros((90*90,1), dtype='float32'))
corner_config_contrast['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
model_corners_contrast.load_state_dict(corner_config_contrast)
#model_torch_load.state_dict()

<All keys matched successfully>

In [59]:

train_df = pd.read_csv("train.csv")
trainingData = CustomMnistDataset_OL(train_df)
train_dataloader = DataLoader(trainingData, batch_size=1, shuffle=True)
data = next(iter(train_dataloader))
X_0, _ = data
data = next(iter(train_dataloader))
X_1, _ = data

In [ ]:
# create brightness values
brightness_range = np.asarray(np.linspace(0, 1, 11), 'float32')
brightness_min = brightness_range[:-1]
brightness_max = brightness_range[1:]

brightness_center = (brightness_min+brightness_max)/2.
eps_brightness = ((brightness_max-brightness_min)/2.).max()

X_brightness = torch.Tensor(brightness_center[:,None])

In [ ]:
# create brightness and contrast values
contrast_range = np.asarray(np.linspace(0, 0.1, 11), 'float32')
contrast_min = contrast_range[:-1]
contrast_max = contrast_range[1:]

contrast_center = (contrast_min+contrast_max)/2.
eps_contrast = ((contrast_max-contrast_min)/2.).max()

X_contrast = torch.Tensor(contrast_center[:,None])

In [94]:
def set_brightness(model, image):
    # create a pytorch model for a specific flatten image
    # once a model has been set for one images, you can call it for any brightness value
    image_flatten = image.view((90*90,))
    brightness_config = model.state_dict()
    brightness_config['linear_perturbation.bias']=image_flatten
    brightness_config['linear_perturbation.weight']=torch.Tensor(np.ones((90*90,1), dtype='float32'))
    model.load_state_dict(brightness_config)
    
def set_contrast(model, image):
    print(torch.Tensor(np.zeros((90*90,), dtype='float32')).shape)
    
    # create a pytorch model for a specific flatten image
    # once a model has been set for one images, you can call it for any contrast value
    image_flatten = image.view((90*90,1))
    print(image_flatten.shape)
    contrast_config = model.state_dict()
    contrast_config['linear_perturbation.bias']=torch.Tensor(np.zeros((90*90,), dtype='float32'))
    contrast_config['linear_perturbation.weight']=image_flatten
    model.load_state_dict(contrast_config)

In [95]:
set_contrast(model_corners_contrast, X_0)

torch.Size([8100])
torch.Size([8100, 1])


In [46]:
# create brightness values
brightness_range = np.asarray(np.linspace(0, 1, 11), 'float32')
brightness_min = brightness_range[:-1]
brightness_max = brightness_range[1:]

brightness_center = (brightness_min+brightness_max)/2.
eps_brightness = ((brightness_max-brightness_min)/2.).max()

X_brightness = torch.Tensor(brightness_center[:,None])

In [47]:
# create brightness and contrast values
contrast_range = np.asarray(np.linspace(0, 0.1, 11), 'float32')
contrast_min = contrast_range[:-1]
contrast_max = contrast_range[1:]

contrast_center = (contrast_min+contrast_max)/2.
eps_contrast = ((contrast_max-contrast_min)/2.).max()

X_contrast = torch.Tensor(contrast_center[:,None])

In [48]:
# before
model_digit(X_brightness)[0]

tensor([-10.0793,  16.3645,  -1.0624,  -6.1694,  -2.2533,  -0.0239,  -1.3484,
         -1.0524,  -4.4216,  -1.7593], grad_fn=<SelectBackward0>)

In [49]:
set_brightness(model_digit, X_0)

In [50]:
# after
model_digit(X_brightness)[0]

tensor([ 14.6401, -14.0441,   1.6154,  -8.7187,  -2.7874,  -3.2572,   6.3042,
        -10.6056,   6.3706,   1.3714], grad_fn=<SelectBackward0>)

In [61]:
model_corners(X_contrast)[0]

tensor([36.8423, 37.8334, 62.9065, 64.0585], grad_fn=<SelectBackward0>)

torch.Size([8100, 1])
torch.Size([8100])


RuntimeError: Error(s) in loading state_dict for NeuralNetwork_BrightnessContrast:
	size mismatch for linear_perturbation.weight: copying a param with shape torch.Size([8100]) from checkpoint, the shape in current model is torch.Size([8100, 1]).
	size mismatch for linear_perturbation.bias: copying a param with shape torch.Size([8100, 1]) from checkpoint, the shape in current model is torch.Size([8100]).